The following comes from [Use Word Embedding Layers - Deep Learning Keras](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/).
# Training Your Own Embeddings

In [17]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [ ]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels

# integer encode the documents


In [6]:
# pad documents to a max length of 4 words




[[44 10  0  0]
 [33 41  0  0]
 [43 30  0  0]
 [40 41  0  0]
 [49  0  0  0]
 [41  0  0  0]
 [36 30  0  0]
 [ 7 33  0  0]
 [36 41  0  0]
 [40 44 10 30]]


In [8]:
# define the model





In [1]:
# compile the model

# summarize the model


In [2]:
# fit the model


# evaluate the model



# Using Pre-Trained Embeddings

In [3]:

# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels


# prepare tokenizer



# integer encode the documents


# pad documents to a max length of 4 words



# load the whole embedding into memory


In [13]:
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [14]:
# create a weight matrix for words in training docs






In [15]:
# define model





In [16]:
# compile the model


# summarize the model

# fit the model

# evaluate the model



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_4 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None
Accuracy: 100.000000
